1. Carregar e Pré-processar o Áudio
Converter o arquivo para um formato adequado (wav com taxa de amostragem fixa).
Normalizar o áudio e converter para mono.
Opcionalmente, aplicar filtragem para remoção de ruídos.

In [2]:
from pydub import AudioSegment
import librosa

def load_audio(file_path):
    audio = AudioSegment.from_file(file_path).set_channels(1).set_frame_rate(44100)
    samples = audio.get_array_of_samples()
    return samples, audio.frame_rate
    

ModuleNotFoundError: No module named 'pydub'

In [6]:
# Provide the correct path to the file
samples, frame_rate = load_audio('wav_musics\\input.wav')

print("samples length: ", len(samples))
print("frame_rate: ", frame_rate)

samples length:  8740864
frame_rate:  44100


### Converter WAV para MIDI

In [7]:
from basic_pitch.inference import predict_and_save
import os

def convert_wav_to_midi(wav_file, output_midi):
    predict_and_save([wav_file], output_midi)

# Exemplo de uso:
convert_wav_to_midi("input.wav", "output.mid")


ModuleNotFoundError: No module named 'basic_pitch'

4. Extrair Frequências Dominantes
Utilizar a Transformada de Fourier (FFT) para obter a distribuição espectral do áudio.
Identificar os componentes mais relevantes, separando melodia, baixo e percussão.

In [15]:
import numpy as np
import librosa.display
import matplotlib.pyplot as plt

def compute_fft(samples, sr):
    stft = np.abs(librosa.stft(samples, n_fft=2048, hop_length=512))
    freqs = librosa.fft_frequencies(sr=sr)
    
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(librosa.amplitude_to_db(stft, ref=np.max),
                             y_axis='log', x_axis='time')
    plt.colorbar()
    plt.title("Espectrograma")
    plt.show()
    
    return freqs, stft

5. Detectar Notas Musicais
Utilizar algoritmos de pitch detection (Autocorrelação, HPS) para extrair notas.
Converter as frequências em notas MIDI.

In [16]:
def pitch_detection(samples, sr):
    pitches, magnitudes = librosa.piptrack(y=samples, sr=sr)
    pitch_values = np.max(pitches, axis=0)
    return pitch_values

6. Reduzir para 4 Canais
Aplicar clustering (K-means) para agrupar as frequências mais relevantes em 4 categorias:
Canal 1 e 2: Ondas pulsadas (melodia/harmonia)
Canal 3: Onda triangular (baixo)
Canal 4: Ruído (percussão)

In [17]:
from sklearn.cluster import KMeans

def reduce_polyphony(frequencies):
    kmeans = KMeans(n_clusters=4, random_state=0).fit(frequencies.reshape(-1, 1))
    return kmeans.labels_

7. Criar Arquivo MIDI
Gerar um arquivo MIDI compatível com os 4 canais do NES.

In [18]:
from mido import Message, MidiFile, MidiTrack

def create_midi(notes, output_file):
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)

    for note in notes:
        track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=480))

    midi.save(output_file)

8. Ajustar Timing (Quantização Temporal)
Ajustar a duração das notas conforme o clock do NES.
Reduzir notas rápidas para evitar sobrecarga.

9. Testar e Ajustar a Ferramenta
Comparar a saída MIDI com a música original.
Aplicar filtros para melhorar a transcrição.
Verificar a fidelidade da conversão e ajustar o algoritmo.

10. Criar Interface (Opcional)
Criar uma interface simples em Python para o usuário carregar arquivos e baixar a conversão.